# Voxel VAE-GAN Training

This notebook is designed to provide a wholistic vae-gan training experience. You can adjust the model and training parameters through the sacred configuration file, you can view training progress in tensorboard, and you can (wip) create reconstructions with the saved models!

References:

* https://github.com/anitan0925/vaegan/blob/master/examples/train.py
  * Runs 20 epochs on separate VAE and GAN then 200 on VAEGAN
* https://github.com/jlindsey15/VAEGAN/blob/master/main.py
  * Almost clear code for vaegan paper
* https://arxiv.org/pdf/1512.09300.pdf
  * vaegan paper
* https://github.com/timsainb/Tensorflow-MultiGPU-VAE-GAN
  * Best code yet!
  

## Setup

In [1]:
import env
from train_vaegan import train_vaegan
from data.thingi10k import Thingi10k
from data.modelnet10 import ModelNet10
from data import MODELNET10_TOILET_INDEX, MODELNET10_SOFA_INDEX, MODELNET10_SOFA_TOILET_INDEX
from models import MODEL_DIR


# plot things
%matplotlib inline
# autoreload modules
%load_ext autoreload
%autoreload 2

## Prepare Sacred Experiment

In [2]:
from sacred.observers import FileStorageObserver
from sacred import Experiment
import os

ex = Experiment(name='voxel_vaegan_notebook', interactive=True)
ex.observers.append(FileStorageObserver.create('experiments_vaegan'))

@ex.main
def run_experiment(cfg):
    train_vaegan(cfg)

import datetime
last_model_dir = None

## Prepare Model Config

The model dir is generated with a timestamp. This keeps you from overwriting past results and keeps results separate to avoid confusing tensorboard.

But be warned! These model dirs can take up space, so you might need to periodically go back and delete ones you do not care about.

Also, if you ever train a model that you would really like to keep, I recommend moving it to a new directory with a special name like "best_model_ever".

In [3]:
DATASET_CLASS = 'ModelNet10'
#INDEX = MODELNET10_SOFA_TOILET_INDEX
#INDEX = MODELNET10_SOFA_INDEX
INDEX = MODELNET10_TOILET_INDEX

def make_cfg():
    model_dir = os.path.join(
        MODEL_DIR,
        'voxel_vaegan1/modelnet10/{}'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')))
    print(model_dir)
    os.makedirs(model_dir)

    cfg = {
        'cfg': {
            "dataset": {
                "class": DATASET_CLASS,
                "index": INDEX,
                #"tag": "animal",
                #"filter_id": 126660,
                #"pctile": 1.0,
                "splits": True
                #"splits": {
                #    "train": .8,
                #    "dev": .1,
                #    "test": .1
                #}
            },
            "generator": {
                "verbose": True,
                "pad": True
            }, 
            "model": {
                "ckpt_dir": model_dir,
                "voxels_dim": 32,
                "batch_size": 32,
                # Do 0.0001 for 1 epoch, then 0.001 for rest of training
                #"learning_rate": [(1, 0.0001), (None, 0.001)],
                #"learning_rate": 0.0001,
                "enc_lr": 0.0001,
                "dec_lr": 0.0001,
                "dis_lr": 0.0001,
                "epochs": 201,
                "keep_prob": 0.8,
                "kl_div_loss_weight": 100,
                "recon_loss_weight": 10000,
                "ll_weight": 0.001,
                "dec_weight": 100,
                "latent_dim": 100,
                "verbose": True,
                "debug": False,
                "input_repeats": 1,
                "display_step": 6000,
                #"example_stl_id": 126660,
                "voxel_prob_threshold": 0.065,
                "dev_step": 10,
                "save_step": 10,
                'launch_tensorboard': True,
                'tb_dir': 'tb',
                #'tb_compare': [('best_sofa_and_toilet', '/home/jcworkma/jack/3d-form/models/voxel_vaegan1/modelnet10/2019-03-15_17-08-43/tb')],
                'tb_compare': [('best_vaegan', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-17_08-40-29/tb')],
                'no_gan': False
            }
        }
    }
    
    return cfg

## Tensorboard Prep

We launch tensorboard with a call to the python subprocess module. Sometimes, that process does not die with the rest of the experiment and lingers on as a system process. This becomes a problem when we try to initialize tensorboard for the next experiment because they cannot share the same port!

The function below is designed to solve this problem. It uses the linux pgrep utility to search for existing tensorboard processes and kill them. Note that this probably won't work on Windows.

In [4]:
from utils import kill_tensorboard

kill_tensorboard()

['pgrep', 'tensorboard'] yielded -> b'11784\n'
killed b'11784'!


## Training

We start with a check that we are not attempting to overwrite the last MODEL_DIR. If you are blocked by the assert, re-execute the cfg code above to generate a new MODEL_DIR. This will allow you to move ahead with training.

The sacred experiment will save away a copy of your experiment settings in an experiments directory. This can be accessed later in case we need to retrieve a prime config.

If tensorboard is enabled, tune in at localhost:6006 or your_ip:6006
   

In [5]:
cfg = make_cfg()
model_dir = cfg.get('cfg').get('model').get('ckpt_dir')
kill_tensorboard()

/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-17_08-40-29
['pgrep', 'tensorboard'] yielded -> b''


In [ ]:
if last_model_dir == model_dir:
    print('dont overwrite!')
    assert False
else:
    last_model_dir = model_dir

ex.run(config_updates=cfg)

WARNING - root - Added new config entry: "cfg.dataset.class"
WARNING - root - Added new config entry: "cfg.dataset.index"
WARNING - root - Added new config entry: "cfg.dataset.splits"
WARNING - root - Added new config entry: "cfg.generator.pad"
WARNING - root - Added new config entry: "cfg.generator.verbose"
WARNING - root - Added new config entry: "cfg.model.batch_size"
WARNING - root - Added new config entry: "cfg.model.ckpt_dir"
WARNING - root - Added new config entry: "cfg.model.debug"
WARNING - root - Added new config entry: "cfg.model.dec_lr"
WARNING - root - Added new config entry: "cfg.model.dec_weight"
WARNING - root - Added new config entry: "cfg.model.dev_step"
WARNING - root - Added new config entry: "cfg.model.dis_lr"
WARNING - root - Added new config entry: "cfg.model.display_step"
WARNING - root - Added new config entry: "cfg.model.enc_lr"
WARNING - root - Added new config entry: "cfg.model.epochs"
WARNING - root - Added new config entry: "cfg.model.input_repeats"
WARNIN

Logging to /home/jcworkma/jack/3d-form/src/logs/2019-03-17_08-40__root.log
Starting train_vaegan main
Numpy random seed: 288585083
Saved cfg: /home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-17_08-40-29/cfg.json
Dataset: <class 'data.modelnet10.ModelNet10'>
Using dataset index /home/jcworkma/jack/3d-form/src/../data/processed/modelnet10_toilet_index.csv and pctile None
Shuffling dataset
dataset n_input=7104
Splitting Datasets
Num input = 7104
Num batches per epoch = 222.00
Initializing VoxelVaegan
encoder/enc_conv1/batchnorm/add_1:0: (?, 30, 30, 30, 8)
encoder/batch_normalization/batchnorm/add_1:0: (?, 15, 15, 15, 16)
encoder/batch_normalization_1/batchnorm/add_1:0: (?, 13, 13, 13, 32)
encoder/batch_normalization_2/batchnorm/add_1:0: (?, 7, 7, 7, 64)
encoder/batch_normalization_3/batchnorm/add_1:0: (?, 7, 7, 7, 343)
encoder/batch_normalization_4/batchnorm/add_1:0: (?, 100)
encoder/batch_normalization_5/batchnorm/add_1:0: (?, 100)
epsilon: (?, 100)
z: (?, 100)


Epoch: 0 / 50, Batch: 26 (0 / 864), Elapsed time: 1.83 mins
Enc Loss = 54.91, KL Divergence = 0.44, Reconstruction Loss = 0.11, ll_loss = 11035.91, dis_Loss = 5.35, dec_Loss = 80.32, Elapsed time: 1.90 mins
Epoch: 0 / 50, Batch: 27 (0 / 896), Elapsed time: 1.90 mins
Enc Loss = 56.39, KL Divergence = 0.43, Reconstruction Loss = 0.11, ll_loss = 13562.69, dis_Loss = 5.63, dec_Loss = 82.85, Elapsed time: 1.97 mins
Epoch: 0 / 50, Batch: 28 (0 / 928), Elapsed time: 1.97 mins
Enc Loss = 52.43, KL Divergence = 0.42, Reconstruction Loss = 0.11, ll_loss = 10464.23, dis_Loss = 6.16, dec_Loss = 79.75, Elapsed time: 2.04 mins
Epoch: 0 / 50, Batch: 29 (0 / 960), Elapsed time: 2.04 mins
Enc Loss = 56.81, KL Divergence = 0.43, Reconstruction Loss = 0.11, ll_loss = 13418.10, dis_Loss = 5.61, dec_Loss = 82.70, Elapsed time: 2.11 mins
Epoch: 0 / 50, Batch: 30 (0 / 992), Elapsed time: 2.11 mins
Enc Loss = 57.15, KL Divergence = 0.43, Reconstruction Loss = 0.12, ll_loss = 14036.16, dis_Loss = 6.04, dec_Los

Epoch: 0 / 50, Batch: 66 (0 / 2144), Elapsed time: 4.59 mins
Enc Loss = 44.86, KL Divergence = 0.32, Reconstruction Loss = 0.11, ll_loss = 12552.04, dis_Loss = 5.66, dec_Loss = 81.79, Elapsed time: 4.66 mins
Epoch: 0 / 50, Batch: 67 (0 / 2176), Elapsed time: 4.66 mins
Enc Loss = 45.61, KL Divergence = 0.33, Reconstruction Loss = 0.11, ll_loss = 13035.90, dis_Loss = 5.42, dec_Loss = 82.28, Elapsed time: 4.73 mins
Epoch: 0 / 50, Batch: 68 (0 / 2208), Elapsed time: 4.73 mins
Enc Loss = 47.75, KL Divergence = 0.35, Reconstruction Loss = 0.13, ll_loss = 12546.20, dis_Loss = 4.50, dec_Loss = 81.78, Elapsed time: 4.80 mins
Epoch: 0 / 50, Batch: 69 (0 / 2240), Elapsed time: 4.80 mins
Enc Loss = 45.16, KL Divergence = 0.32, Reconstruction Loss = 0.12, ll_loss = 12843.16, dis_Loss = 4.53, dec_Loss = 82.08, Elapsed time: 4.87 mins
Epoch: 0 / 50, Batch: 70 (0 / 2272), Elapsed time: 4.87 mins
Enc Loss = 39.74, KL Divergence = 0.29, Reconstruction Loss = 0.10, ll_loss = 11080.94, dis_Loss = 5.77, de

Epoch: 0 / 50, Batch: 106 (0 / 3424), Elapsed time: 7.35 mins
Enc Loss = 41.71, KL Divergence = 0.27, Reconstruction Loss = 0.11, ll_loss = 14239.60, dis_Loss = 4.68, dec_Loss = 83.43, Elapsed time: 7.42 mins
Epoch: 0 / 50, Batch: 107 (0 / 3456), Elapsed time: 7.42 mins
Enc Loss = 40.84, KL Divergence = 0.29, Reconstruction Loss = 0.11, ll_loss = 11876.58, dis_Loss = 6.29, dec_Loss = 81.07, Elapsed time: 7.49 mins
Epoch: 0 / 50, Batch: 108 (0 / 3488), Elapsed time: 7.49 mins
Enc Loss = 42.04, KL Divergence = 0.29, Reconstruction Loss = 0.12, ll_loss = 13331.54, dis_Loss = 5.33, dec_Loss = 82.52, Elapsed time: 7.56 mins
Epoch: 0 / 50, Batch: 109 (0 / 3520), Elapsed time: 7.56 mins
Enc Loss = 40.78, KL Divergence = 0.29, Reconstruction Loss = 0.11, ll_loss = 11659.59, dis_Loss = 5.09, dec_Loss = 80.85, Elapsed time: 7.63 mins
Epoch: 0 / 50, Batch: 110 (0 / 3552), Elapsed time: 7.63 mins
Enc Loss = 40.39, KL Divergence = 0.27, Reconstruction Loss = 0.10, ll_loss = 13772.58, dis_Loss = 5.2

...

Enc Loss = 15.76, KL Divergence = 0.03, Reconstruction Loss = 0.06, ll_loss = 12990.26, dis_Loss = 5.03, dec_Loss = 45.43, Elapsed time: 274.04 mins
Enc Loss = 16.63, KL Divergence = 0.03, Reconstruction Loss = 0.06, ll_loss = 13791.37, dis_Loss = 5.38, dec_Loss = 46.28, Elapsed time: 274.06 mins
Enc Loss = 17.95, KL Divergence = 0.03, Reconstruction Loss = 0.06, ll_loss = 15322.36, dis_Loss = 3.62, dec_Loss = 47.83, Elapsed time: 274.07 mins
Enc Loss = 15.72, KL Divergence = 0.03, Reconstruction Loss = 0.06, ll_loss = 13114.56, dis_Loss = 5.97, dec_Loss = 45.61, Elapsed time: 274.09 mins
Enc Loss = 16.05, KL Divergence = 0.03, Reconstruction Loss = 0.06, ll_loss = 13516.83, dis_Loss = 5.32, dec_Loss = 46.00, Elapsed time: 274.11 mins
Enc Loss = 18.38, KL Divergence = 0.03, Reconstruction Loss = 0.06, ll_loss = 15649.54, dis_Loss = 5.48, dec_Loss = 48.15, Elapsed time: 274.12 mins
Enc Loss = 17.44, KL Divergence = 0.03, Reconstruction Loss = 0.06, ll_loss = 14715.23, dis_Loss = 5.23, d

Epoch: 22 / 50, Batch: 15 (512 / 5504), Elapsed time: 275.67 mins
Enc Loss = 17.09, KL Divergence = 0.02, Reconstruction Loss = 0.06, ll_loss = 14975.45, dis_Loss = 4.91, dec_Loss = 46.95, Elapsed time: 275.74 mins
Epoch: 22 / 50, Batch: 16 (544 / 5504), Elapsed time: 275.74 mins
Enc Loss = 15.10, KL Divergence = 0.02, Reconstruction Loss = 0.06, ll_loss = 12908.51, dis_Loss = 4.67, dec_Loss = 44.84, Elapsed time: 275.82 mins
Epoch: 22 / 50, Batch: 17 (576 / 5504), Elapsed time: 275.82 mins
Enc Loss = 15.21, KL Divergence = 0.02, Reconstruction Loss = 0.06, ll_loss = 12990.77, dis_Loss = 4.71, dec_Loss = 44.91, Elapsed time: 275.89 mins
Epoch: 22 / 50, Batch: 18 (608 / 5504), Elapsed time: 275.89 mins
Enc Loss = 17.88, KL Divergence = 0.02, Reconstruction Loss = 0.06, ll_loss = 15630.18, dis_Loss = 5.34, dec_Loss = 47.50, Elapsed time: 275.96 mins
Epoch: 22 / 50, Batch: 19 (640 / 5504), Elapsed time: 275.96 mins
Enc Loss = 16.48, KL Divergence = 0.02, Reconstruction Loss = 0.06, ll_los

Enc Loss = 18.38, KL Divergence = 0.02, Reconstruction Loss = 0.06, ll_loss = 16244.84, dis_Loss = 5.36, dec_Loss = 47.04, Elapsed time: 278.49 mins
Epoch: 22 / 50, Batch: 54 (1760 / 5504), Elapsed time: 278.50 mins
Enc Loss = 14.81, KL Divergence = 0.02, Reconstruction Loss = 0.05, ll_loss = 12721.15, dis_Loss = 4.15, dec_Loss = 43.44, Elapsed time: 278.57 mins
Epoch: 22 / 50, Batch: 55 (1792 / 5504), Elapsed time: 278.57 mins
Enc Loss = 14.06, KL Divergence = 0.02, Reconstruction Loss = 0.06, ll_loss = 12111.48, dis_Loss = 5.27, dec_Loss = 42.83, Elapsed time: 278.64 mins
Epoch: 22 / 50, Batch: 56 (1824 / 5504), Elapsed time: 278.64 mins
Enc Loss = 16.42, KL Divergence = 0.02, Reconstruction Loss = 0.06, ll_loss = 14395.89, dis_Loss = 3.99, dec_Loss = 45.04, Elapsed time: 278.71 mins
Epoch: 22 / 50, Batch: 57 (1856 / 5504), Elapsed time: 278.71 mins
Enc Loss = 16.44, KL Divergence = 0.02, Reconstruction Loss = 0.06, ll_loss = 14321.86, dis_Loss = 4.64, dec_Loss = 44.99, Elapsed time:

Enc Loss = 16.58, KL Divergence = 0.02, Reconstruction Loss = 0.05, ll_loss = 14660.13, dis_Loss = 4.45, dec_Loss = 44.18, Elapsed time: 281.24 mins
Epoch: 22 / 50, Batch: 92 (2976 / 5504), Elapsed time: 281.24 mins
Enc Loss = 18.90, KL Divergence = 0.02, Reconstruction Loss = 0.06, ll_loss = 16686.89, dis_Loss = 5.38, dec_Loss = 46.20, Elapsed time: 281.32 mins
Epoch: 22 / 50, Batch: 93 (3008 / 5504), Elapsed time: 281.32 mins
Enc Loss = 16.35, KL Divergence = 0.02, Reconstruction Loss = 0.05, ll_loss = 14209.39, dis_Loss = 5.11, dec_Loss = 43.73, Elapsed time: 281.39 mins
Epoch: 22 / 50, Batch: 94 (3040 / 5504), Elapsed time: 281.39 mins
Enc Loss = 14.90, KL Divergence = 0.02, Reconstruction Loss = 0.05, ll_loss = 12734.98, dis_Loss = 5.15, dec_Loss = 42.23, Elapsed time: 281.46 mins
Epoch: 22 / 50, Batch: 95 (3072 / 5504), Elapsed time: 281.46 mins
Enc Loss = 15.84, KL Divergence = 0.02, Reconstruction Loss = 0.05, ll_loss = 13734.90, dis_Loss = 4.64, dec_Loss = 43.16, Elapsed time:

Enc Loss = 13.38, KL Divergence = 0.02, Reconstruction Loss = 0.06, ll_loss = 11181.00, dis_Loss = 5.70, dec_Loss = 39.53, Elapsed time: 284.04 mins
Epoch: 22 / 50, Batch: 130 (4192 / 5504), Elapsed time: 284.04 mins
Enc Loss = 16.06, KL Divergence = 0.02, Reconstruction Loss = 0.06, ll_loss = 13930.32, dis_Loss = 4.31, dec_Loss = 42.23, Elapsed time: 284.11 mins
Epoch: 22 / 50, Batch: 131 (4224 / 5504), Elapsed time: 284.11 mins
Enc Loss = 16.19, KL Divergence = 0.02, Reconstruction Loss = 0.05, ll_loss = 13988.49, dis_Loss = 4.73, dec_Loss = 42.29, Elapsed time: 284.18 mins
Epoch: 22 / 50, Batch: 132 (4256 / 5504), Elapsed time: 284.18 mins
Enc Loss = 17.28, KL Divergence = 0.02, Reconstruction Loss = 0.06, ll_loss = 15114.10, dis_Loss = 4.95, dec_Loss = 43.34, Elapsed time: 284.25 mins
Epoch: 22 / 50, Batch: 133 (4288 / 5504), Elapsed time: 284.26 mins
Enc Loss = 17.09, KL Divergence = 0.02, Reconstruction Loss = 0.05, ll_loss = 14974.95, dis_Loss = 4.43, dec_Loss = 43.26, Elapsed t